# Linear Regression

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## *Functions*

In [3]:
#Read CSVs
def get_data():
    """
    Get training, validation and test data
    """
    train = pd.read_csv("./PA1_train.csv")
    dev = pd.read_csv("./PA1_dev.csv")
    test = pd.read_csv("./PA1_test.csv")
    return train,dev,test

#Add bias
def add_bias(df):
    """
    Add dummy variable to control intercept
    """
    df["dummy"] = 1
    return df

#Seperate Features from response
def seperate(df_train):
    """
    input: dataframe
    """
    #Grab all continuous features
    x = df_train.iloc[:,0:-1]
    #Split dates
    x = split_date(x)
    x = x.drop(["date"], axis=1)
    #Grab response y
    y = df_train.iloc[:,-1]
    
    return x,y
    
    
#Add new features [Month, Day, Year]
def split_date(df_train):
    """
    splits date into seperate features
    input: dataframe
    """
    print("Splitting date...")
    for i in range(len(df_train)):
        df_train.loc[i,"month"] = int(df_train.loc[i,"date"].split("/")[0])
        df_train.loc[i,"day"] = int(df_train.loc[i,"date"].split("/")[1])
        df_train.loc[i,"year"] = int(df_train.loc[i,"date"].split("/")[2])
    print("Done")
    return df_train

#Normalize data
def normalize(df1):
    """
    Normalizes feature matrix
    input: feature df
    """
    print("Normalizing...")
    x = (df1 - np.min(df1))/(np.max(df1) - np.min(df1))
    print("DONE")
    return x

#Linear regression function
def linear_regress(x,y,eta,t,lamb):
    """
    x: input/features
    y: opuput
    eta: learning rate
    t: iterations
    lamb: regularization constant
    """
    n = 0
    e = np.zeros(len(y))
    errors = []
    gradient = []
    
    #Initialize weights [w] and predictions [y_hat]
    w = np.zeros(len(x[0]))
    
    #nom = float("inf") 
    while n<t:
    #while nom > 0.5:
        #Initialize gradient for each epoch
        gradient_vector = np.zeros(len(x[0]))
                
        #Traverse through each data point   
        for i in range(len(x)):
            #Predicted value
            y_hat = np.dot(w.T,x[i])

            #Error
            e[i] = ((y[i] - y_hat)**2)
            
            #Regularization
            if np.dot(w.T,w) == 0:
                r = 0
            else:
                r = (np.dot(w.T,w))**0.5
            
            #Traverse through each feature to update corresponding weights
            for j in range(len(x[0])):
                gradient_vector[j] += ((-2)*(y[i] - y_hat)*x[i,j]) + (2*lamb*r)
            
        #Update weights
        w -=  eta*gradient_vector
        #Calculate SSE
        errors.append(sum(e))
        #Norm of gradient
        gradient.append(np.dot(gradient_vector.T,gradient_vector)**0.5)
        
        ####
        print(f'#####Iteration : {n+1}#####')
        print(f'Gradient : {gradient[n]}')
        
        ####
        
        n += 1
        if (np.dot(gradient_vector.T,gradient_vector)**0.5) < 0.5:
            return w,errors,gradient
    
    return w,errors,gradient

In [9]:
#####DATA PREP#####

#load csv
df_train,df_dev,df_test = get_data()
#Drop ID Feature
df_train = df_train.drop("id", axis=1)

#Grab features and Response
x,y = seperate(df_train)

#Normalize continuous features
x_norm_df = normalize(x)

x_norm_df

Splitting date...
Done
Normalizing...
DONE


,dummy,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,day,year
0,NaN,0.09375,0.275862,0.246849,0.005715,0.4,0.0,0.0,0.50,0.555556,...,0.773913,0.0,0.368687,0.761782,0.394979,0.400000,0.011601,0.545455,0.266667,0.0
1,NaN,0.03125,0.275862,0.234244,0.002682,0.0,0.0,0.0,1.00,0.444444,...,0.226087,0.0,0.631313,0.683127,0.112971,0.318584,0.004985,0.545455,0.566667,0.0
2,NaN,0.12500,0.172414,0.134454,0.001471,0.2,0.0,0.0,0.50,0.444444,...,0.017391,0.0,0.722222,0.707093,0.174059,0.226549,0.003837,0.545455,0.200000,0.0
3,NaN,0.09375,0.172414,0.141807,0.005469,0.0,0.0,0.0,0.75,0.444444,...,0.600000,0.0,0.292929,0.515522,0.300418,0.212389,0.011556,0.272727,0.900000,1.0
4,NaN,0.15625,0.241379,0.363445,0.006426,0.0,0.0,0.5,1.00,0.555556,...,0.539130,0.0,0.035354,0.690043,0.335565,0.361062,0.011188,0.727273,0.100000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,0.09375,0.275862,0.231092,0.013111,0.4,0.0,0.0,1.00,0.333333,...,0.504348,0.0,0.292929,0.526621,0.298745,0.353982,0.006746,0.727273,0.833333,0.0
9996,NaN,0.00000,0.068966,0.044118,0.007566,0.0,0.0,0.0,0.50,0.222222,...,0.365217,0.0,0.843434,0.540454,0.169874,0.138053,0.007440,0.454545,0.100000,0.0
9997,NaN,0.06250,0.137931,0.122899,0.004200,0.0,0.0,0.0,1.00,0.333333,...,0.530435,0.0,0.777778,0.966061,0.158159,0.304425,0.008033,0.181818,0.533333,1.0
9998,NaN,0.06250,0.241379,0.157563,0.000804,0.8,0.0,0.0,0.50,0.444444,...,0.939130,0.0,0.515152,0.800708,0.145607,0.217699,0.001188,0.909091,0.566667,0.0


In [4]:
#####DATA PREP#####

#load csv
df_train,df_dev,df_test = get_data()
#Drop ID Feature
df_train = df_train.drop("id", axis=1)

#Grab features and Response
x,y = seperate(df_train)

#Normalize continuous features
x_norm_df = normalize(x)
#Add Bias 
x_norm_df = add_bias(x_norm_df)
x_norm = x_norm_df.values


Splitting date...
Done
Normalizing...
DONE


In [71]:
weights,sse,gradient = linear_regress(x_norm, y, 0.01, 10, 0)

#####Iteration : 1#####
Gradient : 205280.8150291437
#####Iteration : 2#####
Gradient : 140488987.2017579
#####Iteration : 3#####
Gradient : 96308260401.9164
#####Iteration : 4#####
Gradient : 66021444183852.414
#####Iteration : 5#####
Gradient : 4.525916131948607e+16
#####Iteration : 6#####
Gradient : 3.1026156859595555e+19
#####Iteration : 7#####
Gradient : 2.1269117266249084e+22
#####Iteration : 8#####
Gradient : 1.4580450660796138e+25
#####Iteration : 9#####
Gradient : 9.995221654509298e+27
#####Iteration : 10#####
Gradient : 6.851945680348161e+30


In [14]:
len(x_norm[0])

22

In [60]:
#x_norm_df

In [76]:
#plt.plot(gradient)